In [117]:
import os

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset


from sklearn.metrics import roc_curve,roc_auc_score
import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

特徴量を読み込み、学習の前に必要な前処理を行う
1．正規化
2．train、validデータに分け、そして内部もXとｙに分ける
3．NN入力の形に変える

In [46]:
from sklearn.preprocessing import MinMaxScaler

do noramalization to the columns form col index10

In [102]:
scaler = MinMaxScaler()
span=50
df=pd.read_csv(f'features_new_{span}.csv')
df=df.drop(df.columns[[10]],axis=1)#drop the meaningless col, which is the row index
df.index=df.iloc[:,0]
df=df.drop(['Dates'],axis=1)
df.iloc[:, 10:] = scaler.fit_transform(df.iloc[:, 10:])
df=df.drop(df.columns[[0]],axis=1)
df

,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,NAPMPMIIndex_max_50,NAPMPMIIndex_min_50,...,BDIYindex_large_standard_deviation_0.2_50,column_id_max_50,column_id_min_50,column_id_mean_50,column_id_median_50,column_id_std_50,column_id_var_50,column_id_large_standard_deviation_0.05_50,column_id_large_standard_deviation_0.1_50,column_id_large_standard_deviation_0.2_50
Unnamed: 0.1,,,,,,,,,,,,,,,,,,,,,
49,1999.0,6.0,9.0,2.0,49.0,0.0,-0.012284,0.0,53.796,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,1999.0,6.0,10.0,3.0,50.0,0.0,-0.006889,0.0,53.814,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,1999.0,6.0,11.0,4.0,51.0,0.0,0.000000,0.0,53.832,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,1999.0,6.0,14.0,0.0,52.0,0.0,0.000000,0.0,53.850,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,1999.0,6.0,15.0,1.0,53.0,0.0,0.000000,0.0,53.868,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,2022.0,9.0,19.0,0.0,1490.0,0.0,-0.030041,1.0,52.856,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6123,2022.0,9.0,20.0,1.0,1491.0,0.0,-0.030041,1.0,52.852,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6124,2022.0,9.0,21.0,2.0,1492.0,0.0,-0.030041,1.0,52.848,0.671053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


split the features data into trainX/y and validX/y

In [103]:
def split_data(features,batch_size):
    features_y=features.iloc[:,8]
    features_x=features.drop(features.columns[[8,9]],axis=1)
    y_raw=features_y.to_numpy()
    x_raw=features_x.to_numpy()
    data_x=[]
    data_y=[]
    for i in range(len(x_raw)-batch_size):
        data_x.append(x_raw[i:i+batch_size])
        data_y.append(y_raw[i:i+batch_size])
    split_position=4631-span-batch_size+3
    data_x=np.array(data_x)
    data_y=np.array(data_y)
    x_train=data_x[:split_position]
    x_test=data_x[split_position:]
    y_train=data_y[:split_position]
    y_test=data_y[split_position:]
    return [x_train, y_train, x_test, y_test]

In [104]:
batch_size=10
x_train, y_train, x_test, y_test = split_data(df, batch_size)

In [105]:
x_train=x_train.astype(np.float64)
y_train=y_train.astype(np.float64)
x_test=x_test.astype(np.float64)
y_test=y_test.astype(np.float64)

In [106]:
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (4574, 10, 744)
y_train.shape =  (4574, 10)
x_test.shape =  (1494, 10, 744)
y_test.shape =  (1494, 10)


In [107]:
x_test

array([[[2.016e+03, 1.200e+01, 2.000e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.016e+03, 1.200e+01, 2.100e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.016e+03, 1.200e+01, 2.200e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [2.016e+03, 1.200e+01, 2.900e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.016e+03, 1.200e+01, 3.000e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.017e+03, 1.000e+00, 2.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[2.016e+03, 1.200e+01, 2.100e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.016e+03, 1.200e+01, 2.200e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.016e+03, 1.200e+01, 2.300e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [2.016e+03, 1.200e+01, 3.000e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.017e+03, 1.000e+00, 2.000e+00, ..., 0.000e+00, 0.000e+00,
   

処理した特徴量をNNに入力するtensorの形に変える

In [108]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

NNの構造を決める

In [109]:
n_features = 744
hidden_size = 40
output_size = 2
width = 5
epochs = 100
batch_size = 10
test_batch = 1
num_layers = 2

lr = 0.01
save=False
name = 'model_weights1.pth'

In [116]:
class LSTM_model(nn.Module):
    def __init__(self,n_features,width,hidden_size,output_size,num_layers):
        super(LSTM_model,self).__init__()
        self.lstm = nn.LSTM(n_features, hidden_size, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]
        out = self.fc(lstm_out)
        out = self.softmax(out)
        return out

In [111]:
model = LSTM_model(n_features=n_features,width=width,hidden_size=hidden_size,output_size=output_size,num_layers=num_layers).to(device)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [115]:
x_train[0].shape

torch.Size([10, 744])

In [96]:
train_data = TensorDataset(x_train, y_train_lstm)
test_data = TensorDataset(x_test, y_test_lstm)
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader=DataLoader(test_data,batch_size=batch_size)

train_losses = []
test_losses = []

early_stopping_counter=0
patience=10

for epoch in range(epochs):
    model.train()
    train_loss=0
    val_loss=0
    for x_batch, y_batch in train_loader:
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        train_loss += loss.item() * x_batch.size(0)
    train_loss /= len(train_loader.dataset)
    
    

TypeError: linear(): argument 'input' (position 1) must be Tensor, not tuple

In [ ]:
predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train_lstm.detach().numpy()))